### Surface Response Design

Non-linear responses

In [ ]:
%matplotlib widget
# Dependencies
import matplotlib.pyplot as plt
import numpy as np

# Settings
np.set_printoptions(precision=3)

In [ ]:
num = 3
x1 = np.linspace(-1,1,num=num)
x2 = np.linspace(-1,1,num=num)
x1_, x2_ = np.meshgrid(x1, x2)

def measure_response(x_1, x_2, noise=True):
    y =  1.0
    a =  0.5
    b = -0.5
    c =  2.0
    d = -1.5
    e =  1.0
    y_1 = y + a*x_1 + b*x_2 + c*x_1*x_2 + d*x_1*x_1 + e*x_2*x_2
    if noise:
        y_1 += np.random.normal(0, 0.1, y_1.shape)
    return y_1

y1 = measure_response(x1_, x2_, False)

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1,projection='3d')
ax.set_proj_type('ortho')
ax.view_init(30, 150)
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(np.min(y1),np.max(y1))
ax.set_xticks([-1,0,1])
ax.set_yticks([-1,0,1])
ax.set_zticks([np.min(y1),0,np.max(y1)])
ax.grid(False)
ax.set_xlabel("Factor 1")
ax.set_ylabel("Factor 2")
ax.set_zlabel("Response")
ax.plot_wireframe(x1_, x2_, y1*0, linewidth=1, color='k', alpha=0.5)
ax.scatter(x1_, x2_, y1*0, s=15, color='k', alpha=0.5)
ax.plot_wireframe(x1_, x2_, y1, linewidth=1, color='k')
ax.scatter(x1_, x2_, y1, s=15, color='r', alpha=1)

for xi, yi, zi in zip(x1_.flatten(), x2_.flatten(), y1.flatten()):
    ax.plot([xi, xi], [yi, yi], [0, zi], color='k', alpha=0.2, linewidth=1)

In [ ]:
""" Understanding OFAT vs Full Factor """
# OFAT design
x1m = np.r_[np.ones_like(x1)*x1[1], x1]
x2m = np.r_[x2, np.ones_like(x2)*x2[1]]
print("OFAT\n", np.c_[x1m, x2m])
# Full Factor design
x1m = np.repeat(x1, x1.shape[0])
x2m = np.tile(x2, x2.shape[0])
print("Full Factor\n", np.c_[x1m, x2m])

y1m = measure_response(x1m, x2m)

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1,projection='3d')
ax.set_proj_type('ortho')
ax.view_init(30, 150)
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(np.min(y1),np.max(y1))
ax.set_xticks([-1,0,1])
ax.set_yticks([-1,0,1])
ax.set_zticks([np.min(y1),0,np.max(y1)])
ax.grid(False)
ax.set_xlabel("Factor 1")
ax.set_ylabel("Factor 2")
ax.set_zlabel("Response")
ax.scatter(x1m, x2m, y1m, color='c', alpha=1)

In [ ]:
""" Model regression from measured data """
from sklearn.linear_model import LinearRegression

# Design matrix for second-order model
X = np.column_stack((np.ones(x1m.shape),  # intercept
                     x1m,
                     x2m,
                     x1m*x2m,
                     x1m*x1m,
                     x2m*x2m))

# Fit regression
model = LinearRegression(fit_intercept=False)  # intercept already in X
model.fit(X, y1m)
betas = model.coef_

# Output coefficients
coeff_labels = ['β0', 'β1 (x)', 'β2 (y)', 'β3 (xy)', 'β4 (x^2)', 'β5 (y^2)']
for label, coef in zip(coeff_labels, betas):
    print(f'{label}: {coef:.4f}')

x1m_grid, x2m_grid = np.meshgrid(x1m, x2m)
y1m_grid = (betas[0] +
            betas[1]*x1m_grid +
            betas[2]*x2m_grid +
            betas[3]*x1m_grid*x2m_grid +
            betas[4]*x1m_grid*x1m_grid +
            betas[5]*x2m_grid*x2m_grid)

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1,projection='3d')
ax.set_proj_type('ortho')
ax.view_init(30, 150)
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(np.min(y1),np.max(y1))
ax.set_xticks([-1,0,1])
ax.set_yticks([-1,0,1])
ax.set_zticks([np.min(y1),0,np.max(y1)])
ax.grid(False)
ax.set_xlabel("Factor 1")
ax.set_ylabel("Factor 2")
ax.set_zlabel("Response")
ax.plot_wireframe(x1m_grid, x2m_grid, y1m_grid*0, linewidth=1, color='k', alpha=0.5)
ax.scatter(x1m_grid, x2m_grid, y1m_grid*0, s=15, color='k', alpha=0.5)
ax.plot_wireframe(x1m_grid, x2m_grid, y1m_grid, linewidth=1, color='k')
ax.scatter(x1m_grid, x2m_grid, y1m_grid, s=15, color='c', alpha=1)
ax.scatter(x1_, x2_, y1, s=15, color='r', alpha=1)

In [ ]:
import pandas as pd

# Factor levels
A_levels = np.array([-1, 0, 1])
B_levels = np.array([-1, 0, 1])

# Number of levels per factor
n_A = len(A_levels)
n_B = len(B_levels)

# Total runs
n_runs = n_A * n_B

# Create the design matrix columns
# For A: repeat each level for B times
A = np.repeat(A_levels, n_B)
# For B: tile for n_A times
B = np.tile(B_levels, n_A)

# Stack columns into a design matrix
design = np.column_stack((A, B))

# Convert to DataFrame for readability
df = pd.DataFrame(design, columns=['A', 'B'])
display(df)

# Define true effects (arbitrary example)
# y = intercept + A_effect + B_effect + interactions + squares + noise
def simulate_response(A, B):
    return (  1.0
            + 1.5 * A
            - 0.5 * B
            + 2.0 * A * B
            - 1.5 * A * A
            + 1.0 * B * B
            + np.random.normal(0, 0.5))  # noise

# Apply function row-wise
df['y'] = [simulate_response(a, b) for a, b in design]
display(df)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Fit model with all interactions using categorical variables
model = ols('y ~ A + B + A:B + A^2 + B^2', data=df).fit()

# Perform an ANOVA (analysis of variance) (type I sums of squares)
anova_table = sm.stats.anova_lm(model, typ=1)
display(anova_table)

In [ ]:
""" Lack of Fit Test """
# Create a reduced model (without interaction terms, for example)
reduced_model = ols('y ~ A + B + A:B', data=df).fit()
# Perform an ANOVA (analysis of variance) (type I sums of squares)
anova_table_reduced = sm.stats.anova_lm(reduced_model, typ=1)
display(anova_table_reduced)

# Perform ANOVA to compare models
anova_results = sm.stats.anova_lm(reduced_model, model)
print("ANOVA Results between Reduced and Full")
display(anova_results)